In [1]:
##%matplotlib widget
## with %matplotlib notebook: seems to require ipympl as part of environment, either
## part of the conda environment or "pip install ipympl"
## otherwise, does not show ANY plots in note"book, plt.savefig() works
%matplotlib notebook  
##%matplotlib inline    ## --plt.savefig()  works, but re-sizing does NOT


Check the current GPU usage. Please try to be nice!

In [2]:
!nvidia-smi

Tue Apr 13 11:00:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 440.64.00    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN V             Off  | 00000000:03:00.0 Off |                  N/A |
| 28%   31C    P2    32W / 250W |   9749MiB / 12066MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:83:00.0 Off |                    0 |
| N/A   32C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

> **WARNING**: The card numbers here are *not* the same as in CUDA. You have been warned.

This notebook is first attempt to read in track information and use it to predict the KDE used as input to PvFinder.


## Imports

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import time
import torch
import pandas as pd

# Python 3 standard library
from pathlib import Path

##from torchsummary import summary

## Get the helper functions

Add the directory with the model
definitions to the path so we can import from it:

> When you type `import X`,
Python searches `sys.path` for a python
file named `X.py` to import. So we need to add the model directory to the path.

In [4]:
from model.models_mds_G_simon import AllCNN8Layer_Ca_Two_KDE_withPcnn as Model

In [5]:
from model.training_kde import trainNet, select_gpu, Results
device = select_gpu(0)

1 available GPUs (initially using device 0):
  0 TITAN V


In [6]:
model = Model()

In [7]:
ct = 0
for child in model.children():
  print('ct, child = ',ct, "  ", child)
  if ct < 0:
    print("     About to set param.requires_grad=False for ct = ", ct, "params")
    for param in child.parameters():
        # fixing parameters
        param.requires_grad = False 
  ct += 1


ct, child =  0    Softplus(beta=1, threshold=20)
ct, child =  1    LeakyReLU(negative_slope=0.01)
ct, child =  2    LeakyReLU(negative_slope=0.0)
ct, child =  3    Conv1d(2, 25, kernel_size=(25,), stride=(1,), padding=(12,))
ct, child =  4    Conv1d(25, 25, kernel_size=(15,), stride=(1,), padding=(7,))
ct, child =  5    Conv1d(25, 25, kernel_size=(15,), stride=(1,), padding=(7,))
ct, child =  6    Conv1d(25, 25, kernel_size=(15,), stride=(1,), padding=(7,))
ct, child =  7    Conv1d(25, 25, kernel_size=(15,), stride=(1,), padding=(7,))
ct, child =  8    Conv1d(25, 25, kernel_size=(15,), stride=(1,), padding=(7,))
ct, child =  9    Conv1d(25, 1, kernel_size=(5,), stride=(1,), padding=(2,))
ct, child =  10    Dropout(p=0.15, inplace=False)
ct, child =  11    Dropout(p=0.15, inplace=False)
ct, child =  12    Dropout(p=0.15, inplace=False)
ct, child =  13    Dropout(p=0.15, inplace=False)
ct, child =  14    Dropout(p=0.15, inplace=False)
ct, child =  15    Dropout(p=0.15, inplace=False)
ct,

In [8]:
model = model.to(device)

In [9]:
model_dict = model.state_dict()
dict_name = 'ML/18December2020_AllCNN8Layer_withPcnn_50epochs_moreDataIter1_lr_5em7_aymm_15p0/18December2020_AllCNN8Layer_withPcnn_50epochs_moreDataIter1_lr_5em7_aymm_15p0_final.pyt'
pretrained_dict = torch.load(dict_name)
model_dict.update(pretrained_dict) 
model.load_state_dict(pretrained_dict,strict=False)
model.eval()

AllCNN8Layer_Ca_Two_KDE_withPcnn(
  (softplus): Softplus(beta=1, threshold=20)
  (leaky): LeakyReLU(negative_slope=0.01)
  (relu): LeakyReLU(negative_slope=0.0)
  (conv1): Conv1d(2, 25, kernel_size=(25,), stride=(1,), padding=(12,))
  (conv2): Conv1d(25, 25, kernel_size=(15,), stride=(1,), padding=(7,))
  (conv3): Conv1d(25, 25, kernel_size=(15,), stride=(1,), padding=(7,))
  (conv4): Conv1d(25, 25, kernel_size=(15,), stride=(1,), padding=(7,))
  (conv5): Conv1d(25, 25, kernel_size=(15,), stride=(1,), padding=(7,))
  (conv6): Conv1d(25, 25, kernel_size=(15,), stride=(1,), padding=(7,))
  (conv7): Conv1d(25, 1, kernel_size=(5,), stride=(1,), padding=(2,))
  (conv1dropout): Dropout(p=0.15, inplace=False)
  (conv2dropout): Dropout(p=0.15, inplace=False)
  (conv3dropout): Dropout(p=0.15, inplace=False)
  (conv4dropout): Dropout(p=0.15, inplace=False)
  (conv5dropout): Dropout(p=0.15, inplace=False)
  (conv6dropout): Dropout(p=0.15, inplace=False)
  (conv7dropout): Dropout(p=0.15, inplace=F

In [10]:
torch.jit.script(model).save("ML_dir/18December2020_AllCNN8Layer_withPcnn_50epochs_moreDataIter1_lr_5em7_aymm_15p0_final.pt")